In [267]:
import numpy as np
import h5py
import torch
from torch import nn
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from nn import *

In [3]:
fn = "image1-competition.hdf5"

with h5py.File(fn, "r") as f:
    data = np.array(f["data"])
    wns = np.array(f["wns"])
    tissue_mask = np.array(f["tissue_mask"])
    classes = np.array(f["classes"])

In [26]:
data_predict = data[265:465,360:660]
average_spectrum = np.mean(data_predict, axis=(0, 1))
# plt.plot(wns, average_spectrum)

In [9]:
def cosine_distance(a, b):
    return 1 - np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [64]:
X_test = data[475:580, 870:1000, :]
average_spectrum2 = np.mean(X_test, axis=(0, 1))
cosine_distance(average_spectrum2, average_spectrum)

np.float32(0.00020927191)

In [307]:
h, w = data.shape[:2]
xx, yy = np.meshgrid(np.arange(w), np.arange(h))
coords = np.stack((yy, xx), axis=-1)

data_with_coords = np.concatenate([data, coords], axis=-1)

X_test = data_with_coords[475:580, 870:1000, :]
y_test = classes[475:580, 870:1000]

train_data = np.delete(data_with_coords, slice(475, 580), axis=0)
X_train = np.delete(train_data, slice(870, 1000), axis=1)

train_data_y = np.delete(classes, slice(475, 580), axis=0)
y_train = np.delete(train_data_y, slice(870, 1000), axis=1)


C:\Users\matej\AppData\Local\Temp\ipykernel_30748\2332967178.py:5: RuntimeWarning: invalid value encountered in cast
  data_with_coords = np.concatenate([data, coords], axis=-1)


In [308]:
annotated_sel_test = y_test != -1

X_test = X_test[annotated_sel_test, :]
y_test = y_test[annotated_sel_test]


annotated_sel_train = y_train != -1
X_train = X_train[annotated_sel_train]
y_train = y_train[annotated_sel_train]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [291]:
X_train.shape

(31548, 189)

In [400]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(189, 20),
            nn.Tanh(),
            # nn.Dropout(0.3),
            nn.Linear(20, 20),
            nn.Tanh(),
            # nn.Dropout(0.5),
            nn.Linear(20, 6)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)  # or nn.init.kaiming_uniform_
                nn.init.zeros_(m.bias)
    
def train(X, y, model, learning_rate, num_epochs, loss_fn, X_test, y_test, validation=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.005)
    model.train()
    torch.manual_seed(42)

    for epoch in range(num_epochs):
        
        # Forward pass
        preds = model(X)

        # Compute loss
        loss = loss_fn(preds, y)
        
        # Backward pass
        loss.backward()
        
        # Print loss after each epoch
        if (epoch + 1) % 5 == 0:  # Print every 10 epochs (adjust as needed)
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


        # Update model parameters
        optimizer.step()
        
        # Zero gradients for the next step
        optimizer.zero_grad()

        if validation:      
            # Validation step
            model.eval()  # Switch to evaluation mode (disables dropout, batch norm, etc.)
            with torch.no_grad():  # Disable gradient calculation for validation to save memory and computation
                val_preds = model(X_test)
                val_loss = loss_fn(val_preds, y_test)
            
            # Print validation loss after each epoch
            if (epoch + 1) % 5 == 0:  # Print every 10 epochs (adjust as needed)
                print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss.item():.4f}")
            
        model.train() 

    print("Training complete.")

def test(X_test, y_test, model, loss_fn):
    model.eval()  # Set model to evaluation mode (disables dropout, batch norm, etc.)
    
    # Forward pass
    with torch.no_grad():  # No gradient tracking needed for inference
        preds = model(X_test)
    # Convert predictions to class labels (for classification)
    _, predicted = torch.max(preds, 1)
    
    # Calculate accuracy
    correct = (predicted == y_test).sum().item()
    total = y_test.size(0)
    accuracy = 100 * correct / total
    
    # Calculate cross-entropy loss
    print(y_test.shape)
    loss = loss_fn(preds, y_test)
    
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Test Loss (Cross-Entropy): {loss:.4f}")

0.005, 110, 0.005

In [407]:
model = NeuralNetwork()
train(X_train_tensor, y_train_tensor, model, 0.005, 110, nn.CrossEntropyLoss(), X_test_tensor, y_test_tensor, True)

Epoch [5/110], Loss: 1.4176
Epoch [5/110], Validation Loss: 1.5386
Epoch [10/110], Loss: 1.2171
Epoch [10/110], Validation Loss: 1.3593
Epoch [15/110], Loss: 1.0575
Epoch [15/110], Validation Loss: 1.3581
Epoch [20/110], Loss: 0.9183
Epoch [20/110], Validation Loss: 1.4990
Epoch [25/110], Loss: 0.7905
Epoch [25/110], Validation Loss: 1.6492
Epoch [30/110], Loss: 0.6795
Epoch [30/110], Validation Loss: 1.6847
Epoch [35/110], Loss: 0.5886
Epoch [35/110], Validation Loss: 1.5105
Epoch [40/110], Loss: 0.5190
Epoch [40/110], Validation Loss: 1.0697
Epoch [45/110], Loss: 0.4710
Epoch [45/110], Validation Loss: 0.8767
Epoch [50/110], Loss: 0.4336
Epoch [50/110], Validation Loss: 0.7834
Epoch [55/110], Loss: 0.4038
Epoch [55/110], Validation Loss: 0.5769
Epoch [60/110], Loss: 0.3793
Epoch [60/110], Validation Loss: 0.5092
Epoch [65/110], Loss: 0.3593
Epoch [65/110], Validation Loss: 0.4736
Epoch [70/110], Loss: 0.3427
Epoch [70/110], Validation Loss: 0.4411
Epoch [75/110], Loss: 0.3286
Epoch [

In [408]:
test(X_test_tensor, y_test_tensor, model, nn.CrossEntropyLoss())

torch.Size([1200])
Test Accuracy: 91.92%
Test Loss (Cross-Entropy): 0.3321


In [409]:
model = NeuralNetwork()
annotated_sel = classes != -1
X_real = data_with_coords[annotated_sel]
y_real = classes[annotated_sel]

scaler = StandardScaler()
X_real = scaler.fit_transform(X_real)

data_predict = data_with_coords[265:465,360:660]

X_real = torch.tensor(X_real, dtype=torch.float32)
y_real = torch.tensor(y_real, dtype=torch.long)
train(X_real, y_real, model, 0.01, 110, nn.CrossEntropyLoss(), [], [])

Epoch [5/110], Loss: 1.3211
Epoch [10/110], Loss: 1.0689
Epoch [15/110], Loss: 0.8637
Epoch [20/110], Loss: 0.6744
Epoch [25/110], Loss: 0.5452
Epoch [30/110], Loss: 0.4794
Epoch [35/110], Loss: 0.4276
Epoch [40/110], Loss: 0.3872
Epoch [45/110], Loss: 0.3595
Epoch [50/110], Loss: 0.3375
Epoch [55/110], Loss: 0.3176
Epoch [60/110], Loss: 0.3070
Epoch [65/110], Loss: 0.2902
Epoch [70/110], Loss: 0.2797
Epoch [75/110], Loss: 0.2698
Epoch [80/110], Loss: 0.2636
Epoch [85/110], Loss: 0.2559
Epoch [90/110], Loss: 0.2664
Epoch [95/110], Loss: 0.2446
Epoch [100/110], Loss: 0.2410
Epoch [105/110], Loss: 0.2375
Epoch [110/110], Loss: 0.2336
Training complete.


In [410]:
lin_data_predict = data_predict.reshape(-1, data_predict.shape[-1])
lin_data_predict = scaler.transform(lin_data_predict)

lin_data_predict = torch.tensor(lin_data_predict, dtype=torch.float32)
pred = model(lin_data_predict)
pred = torch.softmax(pred, dim=1)
pred = pred.reshape((200, 300, -1)).detach().numpy()
print(pred.shape)

with open("first.npy", "wb") as f:
    np.save(f, pred.astype(np.float32))

(200, 300, 6)
